<a href="https://colab.research.google.com/github/RonneyPetras/ifood-data-analysis-case/blob/main/01_data_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregamento dos dados

## Importação das bibliotecas

In [ ]:
import pandas as pd
import requests
import gzip
import shutil
import os
from google.colab import drive
import tarfile
import json
from tqdm import tqdm

In [ ]:
# Caminhos dos arquivos
pedidos = r'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz' #arquivo vei com um traço a mais na palavra source
usuarios = r'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz'
merchants = r'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz'
ab_text = r'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz'


## Monta o drive

In [ ]:
caminho_gdrive = "/content/drive/MyDrive/ifood_case/ifood-data" # @param {"type":"string"}


In [ ]:
drive.mount('/content/drive')

# Define caminho (substitua SEU_DIRETORIO pelo nome da pasta que você quer usar)
os.makedirs(caminho_gdrive, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Baixar os arquivos e salvar no Google Drive
def baixar_arquivos(url, destino=caminho_gdrive, timeout=30):
    try:
        os.makedirs(destino, exist_ok=True)

        nome_arquivo = os.path.basename(url)
        caminho_completo = os.path.join(destino, nome_arquivo)

        with requests.get(url, stream=True, timeout=timeout) as r:
            with open(caminho_completo, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

        return caminho_completo

    except Exception as e:
        raise Exception(f"Falha ao baixar {url}: {str(e)}")

In [ ]:
def descompactar_gz(arquivo_gz, destino=caminho_gdrive):
    try:
        nome_arquivo_saida = os.path.splitext(os.path.basename(arquivo_gz))[0]  # Remove a extensão .gz
        caminho_completo_saida = os.path.join(destino, nome_arquivo_saida)

        with gzip.open(arquivo_gz, 'rb') as f_in:
            with open(caminho_completo_saida, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        os.remove(arquivo_gz)
        return caminho_completo_saida
    except Exception as e:
        raise Exception(f"Falha ao descompactar {arquivo_gz}: {str(e)}")

In [ ]:
def extrair_tar_gz(arquivo_tar_gz, destino=caminho_gdrive):
    destino = destino or os.path.dirname(arquivo_tar_gz)
    with tarfile.open(arquivo_tar_gz, 'r:gz') as tar:
        primeiro_arquivo = tar.getnames()[1]
        tar.extractall(path=destino)

    os.remove(arquivo_tar_gz)

    return os.path.join(destino, primeiro_arquivo)

In [ ]:
# Baixando os arquivos
pedidos_raw_file = descompactar_gz(baixar_arquivos(pedidos))
usuarios_raw_file = descompactar_gz(baixar_arquivos(usuarios))
merchants_raw_file = descompactar_gz(baixar_arquivos(merchants))
ab_raw_file = extrair_tar_gz(baixar_arquivos(ab_text))

# Tranformando em data Frame. No caso do Order.json vamos transformalo em parquet para facilitar a manipulação e melhorar o uso de memória RAM

In [ ]:
df_usuarios = pd.read_csv(usuarios_raw_file, sep=',')
df_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806156 entries, 0 to 806155
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   customer_id            806156 non-null  object
 1   language               806156 non-null  object
 2   created_at             806156 non-null  object
 3   active                 806156 non-null  bool  
 4   customer_name          806155 non-null  object
 5   customer_phone_area    806156 non-null  int64 
 6   customer_phone_number  806156 non-null  int64 
dtypes: bool(1), int64(2), object(4)
memory usage: 37.7+ MB


In [ ]:
df_restaurantes = pd.read_csv(merchants_raw_file, sep=',')
df_restaurantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7292 entries, 0 to 7291
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   7292 non-null   object 
 1   created_at           7292 non-null   object 
 2   enabled              7292 non-null   bool   
 3   price_range          7292 non-null   int64  
 4   average_ticket       7292 non-null   float64
 5   takeout_time         7292 non-null   int64  
 6   delivery_time        7291 non-null   float64
 7   minimum_order_value  7197 non-null   float64
 8   merchant_zip_code    7292 non-null   int64  
 9   merchant_city        7292 non-null   object 
 10  merchant_state       7292 non-null   object 
 11  merchant_country     7292 non-null   object 
dtypes: bool(1), float64(3), int64(3), object(5)
memory usage: 633.9+ KB


In [ ]:
df_ab_test = pd.read_csv(ab_raw_file, sep=',')
df_ab_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806467 entries, 0 to 806466
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  806466 non-null  object
 1   is_target    806467 non-null  object
dtypes: object(2)
memory usage: 12.3+ MB


In [ ]:
df_ab_test.head()

,customer_id,is_target
0,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target
1,b821aa8372b8e5b82cdc283742757df8c45eecdd72adf4...,control
2,d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc0...,control
3,6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c...,target
4,dad6b7e222bab31c0332b0ccd9fa5dbd147008facd268f...,control


In [ ]:
order_raw_file = r'/content/drive/MyDrive/ifood_case/ifood-data/order.json'

In [ ]:
# Configurações
chunksize = 100_000
relevant_columns = ["order_id", "customer_id", "order_total_amount", "merchant_id"]
output_path = caminho_gdrive + "/order_data.parquet"
temp_dir = "temp_parquet_chunks"  # Pasta para chunks temporários

# Cria diretório temporário
os.makedirs(temp_dir, exist_ok=True)

# Processa cada chunk e salva em arquivos Parquet separados
for i, chunk in tqdm(enumerate(pd.read_json(order_raw_file, lines=True, chunksize=chunksize))):
    chunk[relevant_columns].to_parquet(
        f"{temp_dir}/chunk_{i}.parquet",
        engine="pyarrow",
        compression="snappy",
        index=False
    )

# Combina todos os chunks em um único Parquet
files = [f"{temp_dir}/{f}" for f in os.listdir(temp_dir) if f.endswith(".parquet")]
df_final = pd.concat([pd.read_parquet(file) for file in files], ignore_index=True)
df_final.to_parquet(output_path, engine="pyarrow", compression="snappy")

# Apaga os temp
for file in files:
    os.remove(file)
os.rmdir(temp_dir)

print("Processo concluído! Arquivo final salvo em:", output_path)

37it [05:02,  8.19s/it]


Processo concluído! Arquivo final salvo em: /content/drive/MyDrive/ifood_case/ifood-data/order_data.parquet


In [ ]:
df_pedidos = pd.read_parquet(output_path)
df_pedidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3670826 entries, 0 to 3670825
Data columns (total 4 columns):
 #   Column              Dtype  
---  ------              -----  
 0   order_id            object 
 1   customer_id         object 
 2   order_total_amount  float64
 3   merchant_id         object 
dtypes: float64(1), object(3)
memory usage: 112.0+ MB


In [ ]:
df_pedidos.head()

,order_id,customer_id,order_total_amount,merchant_id
0,ec00f7aec2f1b32bdaf6b359a91b60a1f58ce2cb452fed...,665c8547a9ea092ba84342bfc55103f1e8a5068f0a26a7...,32.8,c2b068129a510b4c4ed10ff16081428d077f6d3be44785...
1,ee5ba707b0938b565107d0c86b8b4325d4941d06fc73ce...,5e35cb26472772592674730e8907e9e22428354fbd19e6...,38.1,355021c61bb358c499d4d6e4647010a43294463d091c45...
2,c4d0992f315f48e2e145afacf302513699147ba43a187e...,56973dab976fe8b32cdeaa0795af4eddba963a5c992535...,50.2,a9ef451e17cb419dce70451a1c9b733d5809a5aa196853...
3,992f50eaded5ffcf1ed2ad615b11bef0f226edac4b3430...,5aeb6d21ff3e8f1c04e437d3cf677bf56a9ae899c381ba...,21.0,2f9f3fd5434960270578ca3ea5360ef8eadd4454f67a5d...
4,a6d507960380b270f9f19876dd53bd0d23eb5dd8410511...,e80f08f1829cdbf06e2d24cf9880b41ee53f602dff8486...,21.0,3e6131f15538a060e898ec02926240ccee9619fd21710d...


# Unindo os arquivos

In [ ]:
# Merge pedidos + grupo A/B
df_pedidos_teste_ab = pd.merge(
    df_pedidos,
    df_ab_test,
    on="customer_id",
    how="left"
)

df_pedidos_teste_ab.to_parquet(caminho_gdrive + "/pedidos_teste_ab.parquet")
df_pedidos_teste_ab.head()

,order_id,customer_id,order_total_amount,merchant_id,is_target
0,ec00f7aec2f1b32bdaf6b359a91b60a1f58ce2cb452fed...,665c8547a9ea092ba84342bfc55103f1e8a5068f0a26a7...,32.8,c2b068129a510b4c4ed10ff16081428d077f6d3be44785...,target
1,ee5ba707b0938b565107d0c86b8b4325d4941d06fc73ce...,5e35cb26472772592674730e8907e9e22428354fbd19e6...,38.1,355021c61bb358c499d4d6e4647010a43294463d091c45...,target
2,c4d0992f315f48e2e145afacf302513699147ba43a187e...,56973dab976fe8b32cdeaa0795af4eddba963a5c992535...,50.2,a9ef451e17cb419dce70451a1c9b733d5809a5aa196853...,control
3,992f50eaded5ffcf1ed2ad615b11bef0f226edac4b3430...,5aeb6d21ff3e8f1c04e437d3cf677bf56a9ae899c381ba...,21.0,2f9f3fd5434960270578ca3ea5360ef8eadd4454f67a5d...,control
4,a6d507960380b270f9f19876dd53bd0d23eb5dd8410511...,e80f08f1829cdbf06e2d24cf9880b41ee53f602dff8486...,21.0,3e6131f15538a060e898ec02926240ccee9619fd21710d...,target


In [ ]:
df_pedidos_teste_ab.describe()

,order_total_amount
count,3.670826e+06
mean,4.790006e+01
std,1.196738e+02
min,0.000000e+00
25%,2.690000e+01
50%,3.990000e+01
75%,5.740000e+01
max,1.387509e+05
